In [1]:
# Various setup, probably mostly unused

from env import TestEnv
import pandas as pd
import json
import os
from conf import LisaLogging
from bart.common.Utils import area_under_curve
from trappy.plotter import plot_trace
from IPython.display import display
from trappy import ILinePlot
from trappy.stats.grammar import Parser
from bart.sched.SchedMultiAssert import SchedMultiAssert
import pandas as pd
from trace import Trace
LisaLogging.setup()
import logging
logging.getLogger('Trace').setLevel(logging.ERROR)
logging.getLogger('Analysis').setLevel(logging.WARNING)
logging.getLogger('EnergyMeter').setLevel(logging.DEBUG)
%matplotlib inline
from platforms.juno_energy import juno_energy
from platforms.pixel_energy import pixel_energy
import tests.eas.generic
from tests.eas.generic import EnergyModelTest
import numpy as np
from IPython.display import display
from wlgen import RTA, Periodic
from executor import Executor

2017-02-07 15:19:40,587 INFO    : root         : Using LISA logging configuration:
2017-02-07 15:19:40,588 INFO    : root         :   /home/brejac01/sources/lisa/logging.conf


In [2]:
# Some nonsense to get caiman to work on Brendan's computer
p = os.getenv('PATH').split(':')
caiman_path = '/opt/ds5_v5.23.0/bin'
if caiman_path not in p:
    p.insert(0, caiman_path)
    os.environ['PATH'] = ':'.join(p)

In [3]:
from platforms.juno_energy import juno_energy

In [4]:
from platforms.hikey_energy import hikey_energy

In [5]:
te = TestEnv(test_conf={
        'modules': ['cgroups'], 
        'ftrace': {
            'events': ['sched_switch', 'cpu_frequency', 'sched_load_avg_cpu', 'sched_load_avg_task']
        }
    }, force_new=True)

2017-02-07 15:19:40,654 INFO    : TestEnv      : Using base path: /home/brejac01/sources/lisa
2017-02-07 15:19:40,655 INFO    : TestEnv      : Loading default (file) target configuration
2017-02-07 15:19:40,655 INFO    : TestEnv      : Loading target configuration [/home/brejac01/sources/lisa/target.config]...
2017-02-07 15:19:40,658 INFO    : TestEnv      : Loading custom (inline) test configuration
2017-02-07 15:19:40,659 INFO    : TestEnv      : External tools using:
2017-02-07 15:19:40,660 INFO    : TestEnv      :    ANDROID_HOME: /work/android-sdk-linux
2017-02-07 15:19:40,660 INFO    : TestEnv      :    CATAPULT_HOME: /home/brejac01/sources/lisa/tools/catapult
2017-02-07 15:19:40,661 INFO    : TestEnv      : Devlib modules to load: ['cpuidle', 'cgroups', 'cpufreq']
2017-02-07 15:19:40,662 INFO    : TestEnv      : Connecting Android target [04D314DF000BC57F]
2017-02-07 15:19:40,663 INFO    : TestEnv      : Connection settings:
2017-02-07 15:19:40,664 INFO    : TestEnv      :    {'

In [6]:
# Some nonsense to make LISA treat HiKey with the respect it deserves
from trappy.stats.Topology import Topology
hikey_topology = Topology(clusters=[[0, 1, 2, 3], [4, 5, 6, 7]])
te.topology = hikey_topology
te.platform['clusters'] = {'big': [0, 1, 2, 3], 'little': [4, 5, 6, 7]}

In [7]:
def spread_evenly(ntasks, ncpus):
    ret = [0 for _ in range(ncpus)]
    for i in range(ntasks):
        ret[i % ncpus] += 1
    return ret

def get_various_packing_placements(ntasks, ncpus):
    max_spread = spread_evenly(ntasks, ncpus)
    
    max_pack = [ntasks] + [0] * (ncpus - 1)
    
    half_pack = [ntasks / 2] + spread_evenly(ntasks - (ntasks / 2), ncpus -1)
    
    return [max_spread, half_pack, max_pack]

In [8]:
def get_everything(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    ntasks = tasks_per_cpu * ncpus
    task_distribs = get_various_packing_placements(ntasks, ncpus)
    
    util_distribs = []
    wloads = []
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)

    # Stagger the workloads so that they prevent shared idle states
    period_s = 10e-3
    stagger_s = 0.7e-3
    delays = np.arange(period_s, step=stagger_s)
    
    if len(delays) > ntasks:
        print "WARNING: not enough tasks to cover period with wakeups"
    
    tasks = {}
    for i in range(ntasks):
        tasks['{}pct_{}'.format(task_pct, i)] = {
            'kind': 'Periodic',
            'params': {
                'duty_cycle_pct': task_pct,
                'period_ms' : period_s * 1e3,
                'delay_s' : delays[i % len(delays)],
                'duration_s': 2
            }
        }
    
    wloads = {
        'forced_pack': {
            'type': 'rt-app',
            'conf': {
                'class': 'profile',
                'params': tasks,
                'cpus': [0, 1, 2, 3]
            }
        },
        'any_cpu': {
            'type': 'rt-app',
            'conf': {
                'class': 'profile',
                'params': tasks,
            }
        }
    }
        
    return task_distribs, util_distribs, wloads

In [9]:
task_distribs, util_distribs, wloads = get_everything(hikey_energy)

In [18]:
executor = Executor(te, {
        'confs': [{
                'tag': 'myconf',
                'flags': ['ftrace', 'freeze_userspace'],
                'cpufreq': {
                    'governor': 'userspace', # No powersave governor in hikey_defconfi
                    'freqs':{
                        0: 208000,
                        4: 208000
                    }
                }
            }],
        'wloads': wloads,
        'iterations': 1
    })

2017-02-07 15:21:48,549 INFO    : Executor     : Loading custom (inline) test configuration
2017-02-07 15:21:48,551 INFO    : Executor     : 
2017-02-07 15:21:48,552 INFO    : Executor     : ################################################################################
2017-02-07 15:21:48,553 INFO    : Executor     : Experiments configuration
2017-02-07 15:21:48,554 INFO    : Executor     : ################################################################################
2017-02-07 15:21:48,555 INFO    : Executor     : Configured to run:
2017-02-07 15:21:48,556 INFO    : Executor     :      1 target configurations:
2017-02-07 15:21:48,557 INFO    : Executor     :       myconf
2017-02-07 15:21:48,558 INFO    : Executor     :      2 workloads (1 iterations each)
2017-02-07 15:21:48,559 INFO    : Executor     :       any_cpu, forced_pack
2017-02-07 15:21:48,560 INFO    : Executor     : Total: 2 experiments
2017-02-07 15:21:48,561 INFO    : Executor     : Results will be collected under:


In [19]:
executor.run()

2017-02-07 15:21:48,787 INFO    : Executor     : 
2017-02-07 15:21:48,789 INFO    : Executor     : ################################################################################
2017-02-07 15:21:48,790 INFO    : Executor     : Experiments execution
2017-02-07 15:21:48,791 INFO    : Executor     : ################################################################################
2017-02-07 15:21:48,792 INFO    : Executor     : 
2017-02-07 15:21:48,793 INFO    : Executor     : ================================================================================
2017-02-07 15:21:48,794 INFO    : Executor     : configuring target for [myconf] experiments
2017-02-07 15:21:49,183 WARNING : Executor     : Setting target SELinux in permissive mode
2017-02-07 15:21:49,354 INFO    : Executor     : Configuring all CPUs to use [userspace] cpufreq governor
2017-02-07 15:21:49,529 INFO    : Executor     :        CPUFreq - CPU frequencies: {0: 208000, 4: 208000}
2017-02-07 15:21:51,797 INFO    : Workload 

HIGHWAY TO THE DANGERZONE
TRYING TO FREEZE


2017-02-07 15:22:19,245 WARNING : Executor     : FTrace events collection enabled


SUCCESSED



2017-02-07 15:22:29,705 DEBUG   : EnergyMeter  : RESET: []
2017-02-07 15:22:29,715 INFO    : Workload     : Workload execution START:
2017-02-07 15:22:29,717 INFO    : Workload     :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/run_dir/any_cpu_00.json 2>&1
2017-02-07 15:22:42,768 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.002293
2017-02-07 15:23:03,737 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 15:23:03,739 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/1/trace.dat
2017-02-07 15:23:03,740 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 15:23:03,741 INFO    : Executor     :    <res_dir>/rtapp:myconf:any_cpu/1/trace_stat.json
2017-02-07 15:23:03,743 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 15:23:07,224 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 15:23:07,227 INFO    : Workload     : Setup new workload forced

HIGHWAY TO THE DANGERZONE
TRYING TO FREEZE


2017-02-07 15:23:32,665 WARNING : Executor     : FTrace events collection enabled


SUCCESSED



2017-02-07 15:23:43,195 DEBUG   : EnergyMeter  : RESET: []
2017-02-07 15:23:43,205 INFO    : Workload     : Workload execution START:
2017-02-07 15:23:43,207 INFO    : Workload     :    /data/local/tmp/bin/taskset 0xF /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/run_dir/forced_pack_00.json 2>&1
2017-02-07 15:23:56,242 DEBUG   : EnergyMeter  : Energy [             BAT]: 0.002297
2017-02-07 15:24:17,589 INFO    : Executor     : Collected FTrace binary trace:
2017-02-07 15:24:17,591 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/1/trace.dat
2017-02-07 15:24:17,592 INFO    : Executor     : Collected FTrace function profiling:
2017-02-07 15:24:17,593 INFO    : Executor     :    <res_dir>/rtapp:myconf:forced_pack/1/trace_stat.json
2017-02-07 15:24:17,594 INFO    : Executor     : Un-freezing userspace tasks
2017-02-07 15:24:21,183 INFO    : Executor     : --------------------------------------------------------------------------------
2017-02-07 15:24:21,185 INFO  

In [20]:
for experiment in executor.experiments:
    plot_trace(Trace(te.platform, experiment.out_dir, ['sched_switch']).ftrace)

In [21]:
def examine_experiment(experiment):
    trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    ma = SchedMultiAssert(trace.ftrace, hikey_topology, experiment.wload.tasks.keys())
    cluster_conclusions = []
    for cluster in te.topology.get_level('cluster'):
        residencies = [t['residency'] for p, t in ma.getResidency('cluster', cluster, percent=True).iteritems()]
        if all(r > 90 for r in residencies):
            cluster_conclusions.append('PACKED')
        else:
            cluster_conclusions.append(None)
    packed = [i for i, c in enumerate(cluster_conclusions) if c == 'PACKED']
    with open(os.path.join(experiment.out_dir, 'energy.json')) as f:
        energy = json.load(f)
        energy = energy['BAT']
    if not packed:
        print "I don't think we packed onto any cluster"
        return False, energy
        # plot_trace(trace.ftrace)
    elif len(packed) == 1:
        [i] = packed
        print 'I think we packed onto cluster {} ({})'.format(i, te.topology.get_node('cluster', i))
        return True, energy
    else:
        # this code is
        print 'what is this'
        # who even.. who wrote this shit
        print 'i dont even'
        # Microsoft recommends Windows 10 with Microsoft Edge® for a more secure browsing experience
        raise Exception(RuntimeError(GenericBamboozlementComplaintFactory2().generateComplaint('bamboozled!!!')))

In [22]:
previous_energy_records = [[], []]

In [23]:
energy_records = previous_energy_records
for experiment in executor.experiments:
    packed, energy = examine_experiment(experiment)
    energy_records[packed].append(energy)

I don't think we packed onto any cluster
I think we packed onto cluster 0 ([0, 1, 2, 3])


In [24]:
import numpy as np

In [25]:
np.mean(energy_records[0]), np.std(energy_records[0])

(0.0022933650504533922, 0.0)

In [26]:
np.mean(energy_records[1]), np.std(energy_records[1])

(0.0022972354820777322, 0.0)

In [ ]:
energy_records

In [ ]:
traces = [Trace(te.platform, e.out_dir, ['sched_switch', 'cpu_frequency']) for e in executor.experiments]

In [ ]:
te.platform

In [ ]:
freqs = [208000, 432000, 729000, 960000, 1200000]
te.platform['freqs']['big'] = freqs
te.platform['freqs']['little'] = freqs

In [ ]:
trace = traces[0]
trace.analysis.frequency.plotClusterFrequencies()

In [ ]:
te.target.big_core

In [ ]:
trace.analysis.tasks.plotTasks()

In [ ]:
# For android-hikey-4.4
energy_records_hikey_regular = [[0.0007220933633207071,
  0.0007259948168912483,
  0.00073977954186192,
  0.0007320675803349075,
  0.0007652096829866224,
  0.0007528051382064471,
  0.0007435907446651704,
  0.0006940457338712526],
 [0.0006728093519416772,
  0.0007162223819813184,
  0.000712395270479278,
  0.000673710608013742,
  0.0006976346544107044,
  0.0006925454978772343,
  0.0006657587436840893,
  0.0007349576824902922,
  0.0006929229334050849,
  0.0007159691648384068,
  0.0007202090557403527,
  0.0006954405751603721]]

In [ ]:
raise Exception('HALT HALT HALT, HALT AT THE GATES OF THE GRAVEYARD OR SURELY PERISH')

##### 💀💀 The code graveyard 💀💀
*abandon hope all ye who scroll past here, for here is the threshold of the graveyard, whither is sent the code of yore, remembered but not missed, respected but not loved, written by the hand of fearful men and trialled by the silicon of scorned machines*

In [ ]:
def get_task_distribs(nrg_model, cpu=0, task_pct=4):
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    ncpus = len(cluster_node.cpus)
    min_cap = cpu_node.min_capacity
    
    task_cap = int(1024 * (task_pct / 100.))
    
    tasks_per_cpu = int(min_cap / task_cap)
    task_distribs = get_various_packing_placements(tasks_per_cpu * ncpus, ncpus)
    return task_distribs

In [ ]:
def get_util_distribs(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    util_distribs = []
    
    task_cap = int(1024 * (task_pct / 100.))
    cpu_node = nrg_model.cpu_nodes[cpu]
    cluster_node = cpu_node.parent
    
    for task_distrib in task_distribs:
        util_distrib = [0 for _ in range(len(nrg_model.cpus))]
        for i, num_tasks_on_cpu in enumerate(task_distrib):
            util_distrib[cluster_node.cpus[i]] = num_tasks_on_cpu * task_cap
        util_distribs.append(util_distrib)
    return util_distribs

In [ ]:
def get_wloads(nrg_model, task_distribs=None, cpu=0, task_pct=4):
    if task_distribs == None:
        task_distribs = get_task_distribs(nrg_model, cpu, task_pct)
    for task_distrib in 

In [ ]:
util_distribs = get_util_distribs(hikey_energy)
for ud in util_distribs:
    display(hikey_energy.estimate_from_cpu_util(ud))

In [ ]:
hikey_energy.cpu_nodes[0].min_capacity

In [ ]:
def get_cluster_count_df(experiment, clusters, trace=None):
    if trace is None:
        trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    tasks = experiment.wload.tasks.keys()
    pids = [trace.getTaskByName(t)[0] for t in tasks]
    task_cpus = trace.data_frame.task_cpus()[pids]
    # drop consecutive duplicates
    task_cpus = task_cpus[(task_cpus.shift(+1) != task_cpus).any(axis=1)]

    df = pd.DataFrame(columns=[str(c) for c in clusters])
    
    for (time, row_in) in task_cpus.iterrows():
        cluster_task_counts = [0 for c in clusters]
        for task_cpu in row_in:
            if isnan(task_cpu):
                continue
            for i, cluster in enumerate(clusters):
                if int(task_cpu) in cluster:
                    cluster_task_counts[i] += 1
                    break
    
        df.loc[time] = pd.Series(cluster_task_counts, index=df.columns)
        
    return df

def were_tasks_cluster_packed(experiment, clusters, trace=None):
    if trace is None:
        trace = Trace(te.platform, experiment.out_dir, ['sched_switch'])
    cluster_counts = get_cluster_count_df(experiment, clusters, trace)
    

In [ ]:
def get_nrg(model, util):
    nrg = model.estimate_from_cpu_util(util, zero_idle=True)
    return sum(nrg.values())